In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName("FraudDetection").getOrCreate()
# Initialize Spark session

In [7]:
raw_data = spark.read.csv("../data/creditcard.csv", header=True, inferSchema=True)
# Read the raw transaction data
if raw_data:
    print("contains data")
else:
    print("doesnot contain any data")

contains data


In [8]:
non_fraud_data = raw_data.filter(raw_data.Class == 0)
fraud_data = raw_data.filter(raw_data.Class == 1)

In [9]:
# since the difference between fraud and non fraud class is too big.
# we will try to equal the data
import pandas as pd
fraud_df = fraud_data.toPandas()
non_fraud_df = non_fraud_data.toPandas().head(492)
equalized_data = pd.concat([fraud_df, non_fraud_df])

In [12]:
'''
Lets start normalizating the data.
We are going to use sklearn MinMaxScalar because it provides a easy way to normalize it.
'''
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1, 1))
equalized_data['Time'] = scaler.fit_transform(equalized_data['Time'].values.reshape(-1,1))
equalized_data['Amount'] = scaler.fit_transform(equalized_data['Amount'].values.reshape(-1,1))

In [16]:
# when there is a pandas dataframe, we use display.
# when its pyspark dataframe, we use df.show()
display(equalized_data) 

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-0.995233,-2.312227,1.951992,-1.609851,3.997906,-0.522188,-1.426545,-2.537387,1.391657,-2.770089,...,0.517232,-0.035049,-0.465211,0.320198,0.044519,0.177840,0.261145,-0.143276,-1.000000,1
1,-0.994458,-3.043541,-3.157307,1.088463,2.288644,1.359805,-1.064823,0.325574,-0.067794,-0.270953,...,0.661696,0.435477,1.375966,-0.293803,0.279798,-0.145362,-0.252773,0.035764,-0.723618,1
2,-0.947613,-2.303350,1.759247,-0.359745,2.330243,-0.821628,-0.075788,0.562320,-0.399147,-0.238253,...,-0.294166,-0.932391,0.172726,-0.087330,-0.156114,-0.542628,0.039566,-0.153029,-0.874646,1
3,-0.917980,-4.397974,1.358367,-2.592844,2.679787,-1.128131,-1.706536,-3.496197,-0.248778,-0.247768,...,0.573574,0.176968,-0.436207,-0.053502,0.252405,-0.657488,-0.827136,0.849573,-0.969175,1
4,-0.911722,1.234235,3.019740,-4.304597,4.732795,3.624201,-1.357746,1.713445,-0.496358,-1.282858,...,-0.379068,-0.704181,-0.656805,-1.632653,1.488901,0.566797,-0.010016,0.146793,-0.999478,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,-0.995809,-0.711928,1.547423,1.799759,2.489060,0.450497,1.073762,0.309374,0.450854,-1.560107,...,-0.115133,-0.136653,0.052595,-0.327937,-0.531480,0.005533,0.515896,0.250372,-0.999608,0
488,-0.995797,1.106251,0.398625,0.860421,2.388862,-0.365843,-0.192568,-0.121769,0.150876,-0.733271,...,-0.253189,-0.952836,0.181643,0.222462,0.083552,-0.342278,-0.017021,0.020976,-0.990894,0
489,-0.995797,-0.424666,0.554293,1.374234,-0.447451,0.288248,-1.082536,0.969035,-0.347446,-0.109678,...,-0.023296,0.103502,0.024513,0.425008,-0.227746,0.182264,0.076720,-0.098667,-0.978245,0
490,-0.995785,1.261597,0.256346,0.513254,0.619054,-0.472482,-0.989342,0.065431,-0.208781,0.035924,...,-0.257305,-0.739810,0.135646,0.376418,0.220320,0.094706,-0.028559,0.019023,-0.999326,0
